# RocketML Tutorial on distributed object detection on videos

## Import Libraries

In [10]:
from rocketml.io import VideoSet, MongoWriter, Video
from rocketml.feature_extraction import ObjectDetection
from rocketml.pipeline import Pipeline
import sys
import os
import pymongo
from pprint import pprint
import subprocess
from time import time

In [2]:
#Simple Generator-->To clear rerun
def simple_gen():
    for x in range(100):
        yield x
        
x=simple_gen()


In [8]:
num=next(x)#<--generate the next sequence number
num

5

## Defaults


In [12]:
start = time()
filename = "/home/ubuntu/rml-notebooks/home_video_list_4.txt"
# Image Set
vm = VideoSet(filename,resize_width=64,resize_height=64,frame_rate=10)

# Get uri
hname = subprocess.check_output(["hostname"])
mongo_uri = "mongodb://"+hname.strip()+":27017"

# MongoWriter
mw = MongoWriter(uri=mongo_uri,db="rml_collection_video_tf_demo",collection="default_run_"+str(num))

# Object Detection
od = ObjectDetection(model_type="ssd_inception_v2_coco_2017_11_17",model_framework="tensorflow")
#od = ObjectDetection(model_type="yolo_v2",model_framework="darknet",object_detection_models_config=test_location)



# Pipeline
pipeline = Pipeline([od,mw])
pipeline.fit(vm,cores="all",verbose=True)



print("Time for object detection = %0.2f"%(time()-start))

Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_1.MOV is dropped
Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_2.MOV is dropped
Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_3.MOV is dropped
Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_5.MOV is dropped
Video file #/home/ubuntu/s3_mount/SampleVideo_1280x720_10mb.mkv is dropped
Video file #/home/ubuntu/s3_mount/SampleVideo_1280x720_10mb.flv is dropped
Video file #/home/ubuntu/s3_mount/Top Gear - [05x05] - 2004.11.21 [VFUA].avi #large scale experiment is dropped
video
------------------
Cluster Resources
------------------
Nodes        = 1
Sockets/node = 1
Cores/socket = 1
--------------------------
Total Cores    = 1
Total Memory(GB)   = 3.86
--------------------------
Time for object detection = 86.48


## Draw objects on a single video

In [13]:
vid = Video(filename="/home/ubuntu/s3_mount/SampleVideo_1280x720_10mb.mp4")
vid.draw_objects(mw,"/home/ubuntu/s3_mount/default_video_"+str(num)+".mp4")

[u'run_2', u'system.indexes', u'test_run', u'run2', u'test_run_0', u'test_run_1', u'test_run_2', u'test_run_5', u'default_run_4', u'default_run_5']


**Inspect JSON**
--------

In [14]:
import json
with open('/home/ubuntu/test_folder/object_detection/object_detection_models_config.json', 'r') as fp:
    data=json.load(fp)

In [15]:
data['ssd_inception_v2_coco_2017_11_17']

{u'tensorflow': {u'channel_scale_1': 127.5,
  u'channel_scale_2': 127.5,
  u'channel_scale_3': 127.5,
  u'class_names': u'/etc/rocketml.d/object_detection/ssd_inception_v2_coco_2017_11_17_class_names.json',
  u'model_binary': u'/etc/rocketml.d/object_detection/ssd_inception_v2_coco_2017_11_17_frozen_inference_graph.pb',
  u'model_config': u'/etc/rocketml.d/object_detection/ssd_inception_v2_coco_2017_11_17.pbtxt',
  u'scale_factor': 0.00784313725490196,
  u'size_x': 300.0,
  u'size_y': 300.0}}

Modify
=====

**class name**

In [16]:
data['ssd_inception_v2_coco_2017_11_17']['tensorflow']['class_names']=u'/home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/ssd_inception_v2_coco_2017_11_17_class_names.json'

**model binary**

In [17]:
data['ssd_inception_v2_coco_2017_11_17']['tensorflow']['model_binary']=u'/home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/frozen_inference_graph.pb'

**model config**

In [18]:
data['ssd_inception_v2_coco_2017_11_17']['tensorflow']['model_config']=u'/home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/pipeline.config.xml'

**Export JSON**

In [19]:
import json
with open('/home/ubuntu/test_folder/object_detection/object_detection_models_config_test.json', 'w') as fp:
    json.dump(data,fp)

In [20]:
import pandas as pd
#from pprint import pprint
pd.set_option('display.max_colwidth',-1)
from pandas.io.json import json_normalize
dataset=pd.DataFrame.from_dict(json_normalize(data), orient='columns')
columns=['coco_channel_scale_1','coco_channel_scale_2','coco_channel_scale_3','coco_class_names','coco_model_binary','coco_model_config','coco_scale_factor','coco_size_x','coco_size_y','yolo_channel_scale_1','yolo_channel_scale_2','yolo_channel_scale_3','yolo_class_names','yolo_model_binary','yolo_model_config','yolo_scale_factor','yolo_size_x','yolo_size_y']
dataset.columns=columns
dataset=dataset.transpose()
dataset.rename(index=str,columns={0:'Values'},inplace=True)

In [21]:
dataset

,Values
coco_channel_scale_1,127.5
coco_channel_scale_2,127.5
coco_channel_scale_3,127.5
coco_class_names,/home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/ssd_inception_v2_coco_2017_11_17_class_names.json
coco_model_binary,/home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/frozen_inference_graph.pb
coco_model_config,/home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/pipeline.config.xml
coco_scale_factor,0.00784314
coco_size_x,300
coco_size_y,300
yolo_channel_scale_1,0


In [27]:
pprint(dataset.iloc[3])

Values    /home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/ssd_inception_v2_coco_2017_11_17_class_names.json
Name: coco_class_names, dtype: object


In [28]:
pprint(dataset.iloc[4])

Values    /home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/frozen_inference_graph.pb
Name: coco_model_binary, dtype: object


In [29]:
pprint(dataset.iloc[5])

Values    /home/ubuntu/test_folder/object_detection/ssd_mobilenet_v1_coco_2018_01_2018/pipeline.config.xml
Name: coco_model_config, dtype: object


## Run Modified

In [22]:
test_location='/home/ubuntu/test_folder/object_detection/object_detection_models_config_test.json'
start = time()
filename = "/home/ubuntu/rml-notebooks/home_video_list_4.txt"
# Image Set
vm = VideoSet(filename,resize_width=64,resize_height=64,frame_rate=10)

# Get uri
hname = subprocess.check_output(["hostname"])
mongo_uri = "mongodb://"+hname.strip()+":27017"

# MongoWriter
mw = MongoWriter(uri=mongo_uri,db="rml_collection_video_tf_demo",collection="test_run_"+str(num))

# Object Detection
od = ObjectDetection(model_type="ssd_inception_v2_coco_2017_11_17",model_framework="tensorflow")
#od = ObjectDetection(model_type="yolo_v2",model_framework="darknet",object_detection_models_config=test_location)



# Pipeline
pipeline = Pipeline([od,mw])
pipeline.fit(vm,cores="all",verbose=True)


print('Test Run:',num)
print("Time for object detection = %0.2f"%(time()-start))

Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_1.MOV is dropped
Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_2.MOV is dropped
Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_3.MOV is dropped
Video file #/home/ubuntu/rocketml-dev/data/home-videos/santi_lr_5.MOV is dropped
Video file #/home/ubuntu/s3_mount/SampleVideo_1280x720_10mb.mkv is dropped
Video file #/home/ubuntu/s3_mount/SampleVideo_1280x720_10mb.flv is dropped
Video file #/home/ubuntu/s3_mount/Top Gear - [05x05] - 2004.11.21 [VFUA].avi #large scale experiment is dropped
video
------------------
Cluster Resources
------------------
Nodes        = 1
Sockets/node = 1
Cores/socket = 1
--------------------------
Total Cores    = 1
Total Memory(GB)   = 3.86
--------------------------
('Test Run:', 5)
Time for object detection = 86.43


## Draw objects on a single video

In [23]:
vid = Video(filename="/home/ubuntu/s3_mount/SampleVideo_1280x720_10mb.mp4")
vid.draw_objects(mw,"/home/ubuntu/s3_mount/test_video_"+str(num)+".mp4")

[u'run_2', u'system.indexes', u'test_run', u'run2', u'test_run_0', u'test_run_1', u'test_run_2', u'test_run_5', u'default_run_4', u'default_run_5']


In [55]:
import pymongo
from pymongo import MongoClient

client = MongoClient()
client = MongoClient(mongo_uri)
db = client.rml_collection_video_tf_demo
collection = db["test_run_"+str(num)]

In [103]:
myList=[]
for i in collection.find():
    myList.append(i) 
print 'The number of indexed items:',len(myList)

The number of indexed items: 312


In [104]:
RequestItem=1

Objects MetaData
----

In [105]:
import pandas as pd
#from pprint import pprint
pd.set_option('display.max_colwidth',-1)
from pandas.io.json import json_normalize
dataset=pd.DataFrame.from_dict(json_normalize(myList[RequestItem]), orient='columns')
heading=dataset['_id']
dataset.drop(columns=['objects','_id'],inplace=True)
dataset=dataset.transpose()
dataset.columns=heading
dataset.reset_index(inplace=True)
dataset['index']=dataset['index'].str.replace('frame_meta_data.','')
dataset.rename(columns={'index':'meta_data'},inplace=True)
dataset

_id,meta_data,5b58ed4458e9ba355e239283
0,file_name,/home/ubuntu/s3_mount/SampleVideo_1280x720_10mb.mp4
1,fps,25
2,frame_id,10
3,height,720
4,row_id,1
5,width,1280


Items found
------

In [106]:
objects=pd.DataFrame.from_dict(json_normalize(myList[RequestItem]['objects']), orient='columns')
names=list(objects['name'])
objects.drop(columns=['name'],inplace=True)
objects=objects.transpose()
objects.columns=names
objects.reset_index(inplace=True)
objects.rename(columns={'index':'found_items'},inplace=True)
objects

,found_items,cow
0,confidence,0.369920
1,height,510.227478
2,id,21.000000
3,width,542.751404
4,x_center,334.666901
5,y_center,307.685822


In [54]:
help(od)

Help on ObjectDetection in module rocketml.feature_extraction.object_detection object:

class ObjectDetection(rocketml.base.Node)
 |  Detects object using Pre-built Deep Learning Models
 |  
 |  Method resolution order:
 |      ObjectDetection
 |      rocketml.base.Node
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model_type='yolo_v2', model_framework='darknet', min_confidence=0.24, dump_json=False, dump_json_prefix='/tmp/jobs_folder/object_detection', object_detection_models_config='/etc/rocketml.d/object_detection/object_detection_models_config.json', **kwargs)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from rocketml.base.Node:
 |  
 |  get_app_info(self, data_type, _type='')
 |      Method to get a list of app names, app data types, and conig files matching this node
 |  
 |  to_dict(self)
 |      Return node as a dictionary
 |  
 |  -------------------------------------------------------